In [0]:
%run ./00_functions_and_libraries

In [0]:
%run ./01_params

In [0]:
spark.conf.set('spark.sql.execution.arrow.enabled', True)
spark.conf.set('spark.sql.execution.arrow.fallback.enabled', False)
spark.conf.set("spark.sql.session.timeZone", "America/New_York")
spark.conf.set(
  params["AzureSASLocation"],
  dbutils.secrets.get(scope=params["AzureSASScope"],key=params["AzureSASKey"])
)
intDataDir = params["intermediateDataDir"]
spark.sql('use {}'.format(params["Database"]))

In [0]:
spark.sql('select * from pcr_master').withColumn("nonnull_count", nonnull_count(col("first_name"), col("last_name"), col("middle_name"), col("address"),
                                                                    col('city'), col('state'), col('zip_code'), col('ssn'), col('county'),
                                                                    col('gender'), col('race'), col('dob'), col('mbi'), 
                                                                    col('age'), col('drivers_license_number'), col('state_issuing_drivers_license')
                                                                   )).createOrReplaceTempView('df')
spark.sql('select cast(nonnull_count as int) as nonnull, count(*) from df group by 1 order by 1 asc').display()

spark.sql('Select * from PCR_master@v61').withColumn("nonnull_count", nonnull_count(col("first_name"), col("last_name"), col("middle_name"), col("address"),
                                                                    col('city'), col('state'), col('zip_code'), col('ssn'), col('county'),
                                                                    col('gender'), col('race'), col('dob'), col('mbi'), 
                                                                    col('age'), col('drivers_license_number'), col('state_issuing_drivers_license')
                                                                   )).createOrReplaceTempView('df_dec')
spark.sql('select cast(nonnull_count as int) as nonnull, count(*) from df_dec group by 1 order by 1 asc').display()


In [0]:
%sql

select 
/*
sum(cnt) as total,
sum(case when a.cnt <=1 then 1 else 0 end) as cnt_singleton,
sum(case when a.cnt = 2 then 1 else 0 end) as cnt_two_match,
sum(case when a.cnt > 2 then 1 else 0 end) as cnt_over_two_match
*/
distinct cnt, count(mp_id)
      from (
            select mp_id, count(patient_id) as cnt
            from pcr_master
            group by 1
            ) a
group by 1

In [0]:
spark.sql('select * from pcr_master').describe('pm_score').display()

In [0]:
spark.sql('select distinct round(pm_score,0) as Score, count(patient_id) as Count from pcr_master where pm_score is not null group by 1 order by 1  asc').display()

In [0]:
%sql
select count(*)
from pcr_master 
where substr(mp_id,2,length(mp_id)) = patient_id
and mp_id in (select distinct mp_id from pcr_master group by 1 having count(*) >1)
;

In [0]:
%sql
select count(*)
from pcr_master 
where substr(mp_id,2,length(mp_id)) <> patient_id
and mp_id in (select distinct mp_id from pcr_master group by 1 having count(*) >1)
;

In [0]:
spark.read.format("delta").load('wasbs://et3@edfrvaproddbrstoragegen2.blob.core.usgovcloudapi.net/prod/archive/pcr_master/10_04_2021_PCRmaster.csv').createOrReplaceTempView('pcr_old')
spark.sql('''
select count(*)
from pcr_old
where substr(mp_id,2,length(mp_id)) <> patient_id
and mp_id in (select distinct mp_id from pcr_old group by 1 having count(*) >1)
''').display()


In [0]:
#Get disqualified for matching Count
spark.sql('select count(*) from pcr_master where er_flag = 0').display()

In [0]:
spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/results_pc_pm").createOrReplaceTempView('mega') #Data for PreFix

df=spark.sql('select cluster_size, mp_id from (select mp_id, count(*) as cluster_size from mega group by 1) a')
df.select(avg("cluster_size"), stddev_pop("cluster_size")).show() #Stats pre Fix

df=spark.sql('select cluster_size, mp_id from (select mp_id, count(*) as cluster_size from pcr_master group by 1) a')
df.select(avg("cluster_size"), stddev_pop("cluster_size")).show() #Stats Post Fix



In [0]:
%sql
use prod;
Select
coalesce(mega.cnt,master.cnt) as Cnt,
ifnull(mega.record_num,0) as mega_num,
ifnull(master.record_num,0) as master_num

from (--mega for pre-mega dissolution clusters
      select cnt, count(mp_id) as record_num 
      from (
            select mp_id, count(*) as cnt 
            from mega 
            group by 1
            ) a
      group by 1
) mega
full outer join (
                select cnt, count(mp_id) as record_num 
                from (
                      select mp_id, count(*) as cnt 
                      from pcr_master
                      group by 1
                      ) a
                group by 1
) master
on mega.cnt=master.cnt
;

In [0]:
%sql
select * from pcr_master where mp_id in (select mp_id from mega group by 1 having count(mp_id)>3000)

In [0]:
spark.sql('''
select * from pcr_master
where patient_id in ('2733993','2735518')''').withColumn("nonnull_count", nonnull_count(col("first_name"), col("last_name"), col("middle_name"), col("address"), col('city'), col('state'), col('zip_code'), col('ssn'), col('county'),col('gender'), col('race'), col('dob'), col('mbi'), col('age'), col('drivers_license_number'), col('state_issuing_drivers_license'))).display()

In [0]:
%sql
select * from pcr_master where patient_id in ('8517572','8563929')

In [0]:
predictions=spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData/delta/GOLD/predictions")
predictions.select('sourceID1','sourceID2','pm_score').createOrReplaceTempView('preds')
spark.sql('''
SELECT mp_id,patient_id1, mp_id2,patient_id2, pm_score from(

select
a.MP_ID,
substr(c.sourceID1,13,length(c.sourceID1)) as patient_id1, 
b.MP_ID as MP_ID2,
substr(c.sourceID2,13,length(c.sourceID2)) as patient_id2,
 c.pm_score,
 a.last_name as ID1_last_name, b.last_name as ID2_last_name,
a.first_name as ID1_first_name, b.first_name as ID2_first_name, 
a.address as ID1_address, b.address as ID2_address, 
a.ssn as ID1_ssn, b.SSN as ID2_SSN,
a.mbi as ID1_mbi,b.mbi as ID2_mbi,
a.gender as ID1_gender, b.gender as ID2_gender,
a.race as ID1_race, b.race as ID2_race,
a.dob as ID1_dob, b.dob as ID2_dob,
a.Age as ID1_age, b.age as ID2_age,
a.state as ID1_state, b.state as ID2_state,
a.zip_code as ID1_zip_code, b.zip_code as ID2_zipcode,
a.city as ID1_city, b.city as ID2_city,
a.county as ID1_county, b.county as ID2_county,
rank() over (partition by a.MP_ID order by c.sourceID1,sourceID2) as ranker

from preds c

inner join pcr_master a
on substr(c.sourceID1,13,length(c.sourceID1)) = a.patient_id
and a.dob is null

inner join pcr_master b
on substr(c.sourceID2,13,length(c.sourceID2)) = b.patient_id
and b.dob is null

where c.pm_score >= 75.0
  and c.pm_score <85.0
--and a.mp_id = b.mp_id
) a
where ranker = 1
limit 100
''').display()

In [0]:
predictions=spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData/delta/GOLD/predictions")
predictions.select('sourceID1','sourceID2','pm_score').createOrReplaceTempView('preds')
spark.sql('''
SELECT mp_id,patient_id1, mp_id2,patient_id2, pm_score from(

select
a.MP_ID,
substr(c.sourceID1,13,length(c.sourceID1)) as patient_id1, 
b.MP_ID as MP_ID2,
substr(c.sourceID2,13,length(c.sourceID2)) as patient_id2,
 c.pm_score,
 a.last_name as ID1_last_name, b.last_name as ID2_last_name,
a.first_name as ID1_first_name, b.first_name as ID2_first_name, 
a.address as ID1_address, b.address as ID2_address, 
a.ssn as ID1_ssn, b.SSN as ID2_SSN,
a.mbi as ID1_mbi,b.mbi as ID2_mbi,
a.gender as ID1_gender, b.gender as ID2_gender,
a.race as ID1_race, b.race as ID2_race,
a.dob as ID1_dob, b.dob as ID2_dob,
a.Age as ID1_age, b.age as ID2_age,
a.state as ID1_state, b.state as ID2_state,
a.zip_code as ID1_zip_code, b.zip_code as ID2_zipcode,
a.city as ID1_city, b.city as ID2_city,
a.county as ID1_county, b.county as ID2_county,
rank() over (partition by a.MP_ID order by c.sourceID1,sourceID2) as ranker

from preds c

inner join pcr_master a
on substr(c.sourceID1,13,length(c.sourceID1)) = a.patient_id
and a.SSN is null

inner join pcr_master b
on substr(c.sourceID2,13,length(c.sourceID2)) = b.patient_id
and b.SSN is null

where c.pm_score >= 75.0
  and c.pm_score <85.0
--and a.mp_id = b.mp_id
) a
where ranker = 1
limit 100
''').display()

In [0]:
predictions=spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData/delta/GOLD/predictions")
predictions.select('sourceID1','sourceID2','pm_score').createOrReplaceTempView('preds')
spark.sql('''
SELECT mp_id,patient_id1, mp_id2,patient_id2, pm_score from(

select
a.MP_ID,
substr(c.sourceID1,13,length(c.sourceID1)) as patient_id1, 
b.MP_ID as MP_ID2,
substr(c.sourceID2,13,length(c.sourceID2)) as patient_id2,
 c.pm_score,
 a.last_name as ID1_last_name, b.last_name as ID2_last_name,
a.first_name as ID1_first_name, b.first_name as ID2_first_name, 
a.address as ID1_address, b.address as ID2_address, 
a.ssn as ID1_ssn, b.SSN as ID2_SSN,
a.mbi as ID1_mbi,b.mbi as ID2_mbi,
a.gender as ID1_gender, b.gender as ID2_gender,
a.race as ID1_race, b.race as ID2_race,
a.dob as ID1_dob, b.dob as ID2_dob,
a.Age as ID1_age, b.age as ID2_age,
a.state as ID1_state, b.state as ID2_state,
a.zip_code as ID1_zip_code, b.zip_code as ID2_zipcode,
a.city as ID1_city, b.city as ID2_city,
a.county as ID1_county, b.county as ID2_county,
rank() over (partition by a.MP_ID order by c.sourceID1,sourceID2) as ranker

from preds c

inner join pcr_master a
on substr(c.sourceID1,13,length(c.sourceID1)) = a.patient_id
and a.MBI is not null

inner join pcr_master b
on substr(c.sourceID2,13,length(c.sourceID2)) = b.patient_id
and b.MBI is not null

where c.pm_score >= 75.0
 and c.pm_score <85.0
--and a.mp_id = b.mp_id
) a
where ranker = 1
limit 100
''').display()

In [0]:
%sql
select a.mp_id, a.patient_id,a.pm_score
from pcr_master a
where a.mp_id in (select mp_id from pcr_master group by 1 having count(mp_id)>1 and count(mp_id)<590 ) 
and substr(a.mp_id,2,length(a.mp_id)) = a.patient_id
limit 100

In [0]:
%sql
select a.mp_id, a.patient_id,a.pm_score
from pcr_master a
where a.mp_id in (select mp_id from pcr_master group by 1 having count(mp_id)>1 and count(mp_id)<590 ) 
and substr(a.mp_id,2,length(a.mp_id)) <> a.patient_id
limit 100

In [0]:
predictions=spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData/delta/GOLD/predictions")
predictions.select('sourceID1','sourceID2','pm_score').createOrReplaceTempView('preds')
spark.sql('''
SELECT mp_id,patient_id1, mp_id2,patient_id2, pm_score from(

select
a.MP_ID,
substr(c.sourceID1,13,length(c.sourceID1)) as patient_id1, 
b.MP_ID as MP_ID2,
substr(c.sourceID2,13,length(c.sourceID2)) as patient_id2,
 c.pm_score,
 a.last_name as ID1_last_name, b.last_name as ID2_last_name,
a.first_name as ID1_first_name, b.first_name as ID2_first_name, 
a.address as ID1_address, b.address as ID2_address, 
a.ssn as ID1_ssn, b.SSN as ID2_SSN,
a.mbi as ID1_mbi,b.mbi as ID2_mbi,
a.gender as ID1_gender, b.gender as ID2_gender,
a.race as ID1_race, b.race as ID2_race,
a.dob as ID1_dob, b.dob as ID2_dob,
a.Age as ID1_age, b.age as ID2_age,
a.state as ID1_state, b.state as ID2_state,
a.zip_code as ID1_zip_code, b.zip_code as ID2_zipcode,
a.city as ID1_city, b.city as ID2_city,
a.county as ID1_county, b.county as ID2_county,
rank() over (partition by a.MP_ID order by c.sourceID1,sourceID2) as ranker

from preds c

inner join pcr_master a
on substr(c.sourceID1,13,length(c.sourceID1)) = a.patient_id


inner join pcr_master b
on substr(c.sourceID2,13,length(c.sourceID2)) = b.patient_id


where c.pm_score < 85.0
and a.mp_id = b.mp_id
  ) a
where ranker = 1
limit 100
''').display()


In [0]:
%sql
select a.mp_id, a.patient_id,a.pm_score
from mega a
where a.mp_id in (select mp_id from mega group by 1 having count(mp_id)>=590 ) 
order by a.mp_id, a.patient_id, a.address

In [0]:
%sql
select mp_id,patient_id, pm_score from pcr_master where mp_id in (
                                     select mp_id 
                                     from pcr_master 
                                     group by 1 
                                     having count(mp_id)>=590 ) 

order by mp_id, patient_id, address

In [0]:
%sql
select mp_id,patient_id, pm_score from pcr_master where mp_id in (
                                     select mp_id 
                                     from pcr_master 
                                     group by 1 
                                     having count(mp_id)<590 and count(mp_id)>=100 ) 

order by mp_id, patient_id, pm_score

In [0]:
"""For a given MP_ID from a megacluster, pull all PCRs sorted by dispatch date, showing some common features, and the difference between one dispatch timestamp and another - For validation purposes - is this mega cluster legitimately linked?"""

window = Window.partitionBy('mp_id').orderBy(col("dispatch_timestamp").asc())
spark.sql(
"""select mp_id,first_name,patient_id,
last_name,
address,
city,
state,
ssn,
DOB,
dispatch_timestamp,
state||'-'||agency_id as agency_id
from pcr_master where mp_id in (select mp_id from pcr_master group by 1 having count(mp_id)>=100) order by mp_id, dispatch_timestamp asc"""
).withColumn("Prior_Dispatch", lag(col("dispatch_timestamp"),1).over(window)).withColumn('Diff_hours',(col("dispatch_timestamp").cast("long") - col('Prior_Dispatch').cast("long"))/3600).createOrReplaceTempView("time")

spark.sql('''select case
    when Diff_hours >= 0 and Diff_hours < 5 then "Less than 5 Hours"
    when Diff_hours >= 5 and Diff_hours < 11 then "5-10 Hours"
    when Diff_hours >= 11 and Diff_hours < 24 then "11-23 Hours"
    when Diff_hours >= 24 and Diff_hours < 49 then "24-48 Hours"
    when Diff_hours >= 49 then "+48 Hours"
    else "First PCR" 
end as Hour_Diffs,
count(distinct patient_id) from time group by 1''').display()


In [0]:
%sql
Select agency_id, mp_id, count(*) from time group by 1,2

In [0]:
"""For a given MP_ID from a megacluster, pull all PCRs sorted by dispatch date, showing some common features, and the difference between one dispatch timestamp and another - For validation purposes - is this mega cluster legitimately linked?"""

window = Window.orderBy("dispatch_timestamp")
spark.sql(
"""select pm_score,mp_id,patient_id,
first_name,
middle_name,
last_name,
DOB,
SSN,
mbi,
address,
dispatch_timestamp
from pcr_master where mp_id = 'M10228301' order by dispatch_timestamp asc"""
).withColumn("Prior_Dispatch", lag(col("dispatch_timestamp"),1).over(window)).withColumn('Diff_hours',(col("dispatch_timestamp").cast("long") - col('Prior_Dispatch').cast("long"))/3600).display()

In [0]:
#Megacluster by Agency ID (post fix only)
spark.sql('select mp_id, count(mp_id), count(patient_id) from pcr_master group by 1 having count(mp_id)>=100  order by 3 desc').display()

In [0]:
spark.read.load(f"/mnt/edfr/et3/prod/HelperData/delta/GOLD/claims_pm").createOrReplaceTempView("claims_pm")
claim_predictions=spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData_claims/delta/GOLD/claim_predictions")
claim_predictions.createOrReplaceTempView('Matches')

In [0]:
%sql

select count(CLM_UNIQ_ID), count(county), count(city) from claims_pm

In [0]:
spark.read.format("delta").load(f"/mnt/edfr/et3/prod/HelperData_claims/delta/GOLD/claims_pm").createOrReplaceTempView('claims_pm')

In [0]:
%sql
select * from matches

In [0]:
%sql
---Look into claims that are predicted as matches, but date is mismatched (in the case of multiple PCRs choose the one closest to the claim date)
select 
a2.pm_score,a2.patient_id,
a.bene_last_name as claim_last_name, b.last_name as pcr_last_name,
a.bene_1st_name as claim_first_name, b.first_name as pcr_first_name, 
a.BENE_LINE_1_ADR as claim_address, b.address as pcr_address, 
a.bene_ssn_num as claim_ssn, b.SSN as pcr_SSN,
a.bene_mbi_id as claim_mbi,b.mbi as pcr_mbi,
a3.BENE_SEX_CD as claim_gender, b.gender as pcr_gender,
a3.bene_race_cd as claim_race, b.race as pcr_race,
a3.bene_brth_dt as claims_dob, b.dob as pcr_dob,
a3.Age as claims_age, b.age as pcr_age,
a3.src_usps_state_cd as claims_state, b.state as pcr_state,
a3.src_zip5_cd as claims_zip, b.zip_code as pcr_zipcode,
a3.city as claims_city, b.city as pcr_city,
a3.county as claims_county, b.county as pcr_county

from claims_master a

INNER JOIN (
            select CLM_UNIQ_ID, 
            patient_id,pm_score 
            from (
                  select *
                  , rank() over (partition by CLM_UNIQ_ID order by abs(datediff(pcr_dispatch_date,clm_from_dt))) as  date_diff_rank
                  from matches 
                  where CLM_UNIQ_ID in (
                                        Select CLM_UNIQ_ID
                                        from Matches
                                        group by 1
                                        having max(prediction) = 1 and max(match) = 0)
                  and prediction=1 
                  and match = 0
            ) a
            where a.date_diff_rank = 1
) a2
on a.CLM_UNIQ_ID = a2.CLM_UNIQ_ID

left join claims_pm a3
on a.CLM_UNIQ_ID = a3.CLM_UNIQ_ID

left join pcr_master b
on a2.patient_id=b.patient_id

In [0]:
distinct
a2.pm_score,a2.patient_id,
a.bene_last_name as claim_last_name, b.last_name as pcr_last_name,
a.bene_1st_name as claim_first_name, b.first_name as pcr_first_name, 
a.BENE_LINE_1_ADR as claim_address, b.address as pcr_address, 
a.bene_ssn_num as claim_ssn, b.SSN as pcr_SSN,
a.bene_mbi_id as claim_mbi,b.mbi as pcr_mbi,
a3.BENE_SEX_CD as claim_gender, b.gender as pcr_gender,
a3.bene_race_cd as claim_race, b.race as pcr_race,
a3.bene_brth_dt as claims_dob, b.dob as pcr_dob,
a3.Age as claims_age, b.age as pcr_age,
a3.src_usps_state_cd as claims_state, b.state as pcr_state,
a3.src_zip5_cd as claims_zip, b.zip_code as pcr_zipcode,
a3.city as claims_city, b.city as pcr_city,
a3.county as claims_county, b.county as pcr_county


left join claims_pm a3
on a.CLM_UNIQ_ID = a3.CLM_UNIQ_ID

left join pcr_master b
on a2.patient_id=b.patient_id

In [0]:
%sql
---Look into claims that are predicted as matches, but date is mismatched (in the case of multiple PCRs choose the one closest to the claim date)
select 
a.bene_last_name as claim_last_name, b.last_name as pcr_last_name,
a.bene_1st_name as claim_first_name, b.first_name as pcr_first_name, 
a.BENE_LINE_1_ADR as claim_address, b.address as pcr_address, 
a.bene_ssn_num as claim_ssn, b.SSN as pcr_SSN,
a.bene_mbi_id as claim_mbi,b.mbi as pcr_mbi,
a3.BENE_SEX_CD as claim_gender, b.gender as pcr_gender,
a3.bene_race_cd as claim_race, b.race as pcr_race,
a3.bene_brth_dt as claims_dob, b.dob as pcr_dob,
a3.Age as claims_age, b.age as pcr_age,
a3.src_usps_state_cd as claims_state, b.state as pcr_state,
a3.src_zip5_cd as claims_zip, b.zip_code as pcr_zipcode,
a3.city as claims_city, b.city as pcr_city,
a3.county as claims_county, b.county as pcr_county

from claims_master a
left join claims_pm a3
on a.CLM_UNIQ_ID = a3.CLM_UNIQ_ID
left join pcr_master b
on a.patient_id=b.patient_id
where a.patient_id is not null

In [0]:
%sql
use prod;
select 
count(*) as total_cnt, 
sum(case when clm_from_dt = clm_thru_dt then 1 else 0 end) as date_check_count,
sum(case when clm_from_dt = clm_submsn_dt then 1 else 0 end) as date_check_count2,
avg(datediff(to_date(clm_submsn_dt, 'ddMMMyyyy'),to_date(clm_thru_dt, 'ddMMMyyyy')))
from claims_master

In [0]:
%sql select * from pcr_master

In [0]:
df_matched = spark.sql("Select * from pcr_master where mp_id in (Select mp_id from pcr_master group by mp_id having count(*)>1)")
df_unmatched = spark.sql("Select * from pcr_master where mp_id not in (Select mp_id from pcr_master group by mp_id having count(*)>1)")
print('***MATCHED COUNTS***')
print('Matched count:',df_matched.count())
print('Matched distinct count:',df_matched.distinct().count())
print('Count MPIDs:',df_matched.select('mp_id').count())
print('Unique MPIDs:',df_matched.select('mp_id').distinct().count())
print('Unique Patient IDs:',df_matched.select('patient_id').distinct().count())

print('***UNMATCHED COUNTS***')
print('Unmatched count:',df_unmatched.count())
print('Unmatched distinct count:',df_unmatched.distinct().count())
print('Count MPIDs:',df_unmatched.select('mp_id').count())
print('Unique MPIDs:',df_unmatched.select('mp_id').distinct().count())
print('Unique Patient IDs:',df_unmatched.select('patient_id').distinct().count())


In [0]:
%sql
select * from pcr_master where patient_id = '12256682'

In [0]:
claim_preds_dir=params["intermediateDataDir_claims"]
spark.read.format("delta").load(f"{claim_preds_dir}/delta/GOLD/claim_predictions").createOrReplaceTempView('matches')
spark.sql('select * from Matches').filter('match==1')\
.withColumn('window',rank().over(Window.partitionBy(col('CLM_UNIQ_ID')).orderBy(col('pm_score').desc(),col('patient_id').desc())))\
.filter('window==1').drop('rank','window','match','prediction')\
.createOrReplaceTempView('Matches')

In [0]:
%sql
select 
a.CLM_UNIQ_ID,
a.mp_ID,
a.patient_id,
a.pm_score,
b.Cluster_Size

from matches a
inner join (select mp_id, count(mp_id) as Cluster_Size from pcr_master group by 1) b
on a.mp_id=b.mp_id

In [0]:
%sql
select 
agency_state||'-'||agency_id as agency_id,
count(patient_id) as Total_Records,
sum(er_flag) as Complete_Records,
count(patient_id)- sum(er_flag) as Incomplete_Records,
sum(er_flag)/count(patient_id) as PCT_Complete

from pcr_master
group by 1
order by 4 desc

In [0]:
%sql
select CLM_UNIQ_ID as claim_id
from claims_master
where patient_id is not null
--and patient_id is not in (select patient_id from)